# [Case Study] XYZ Company Investment

### Author: Raquel Castromonte

# Part 1: Background Information

### **The Client**

XYZ is a private firm in US. Due to remarkable growth in the Cab Industry in last few years and multiple key players in the market, it is planning for an investment in Cab industry and as per their Go-to-Market(G2M) strategy they want to understand the market before taking final decision.

### **Project delivery:**

You have been provided with multiple data sets that contains information on 2 cab companies. Each file (data set) provided represents different aspects of the customer profile. XYZ is interested in using your actionable insights to help them identify the right company to make their investment. The outcome of your delivery will be a presentation to XYZ’s Executive team. 

### **Questions**

**I. Insights**
1. Which company has the most rides all-time (from 2016-2018)?

**A. Demographics**
1. What are the different cities?
2. How many trips by cab company
3. What is the income range of the customers?
4. Gender composition by cab company.
5. City composition.

**B. Profit Analysis**
1. Profit Table Insights
2. Average profit per km
3. Profit by year
4. Profit contribution by income class.
5. Profit contribution by gender.

**C. Precipitation**
1. Which cab service is called more often when it rains?

**Data Information**

Data is taken from 01/01/2016 to 12/31/2018.

| Dataset | Details |
|--------------|-----------------|
| Cab_Data.csv | this file includes details of transaction for 2 cab companies |
| City.csv | this file contains list of US cities, their population and number of cab users |
| Customer_ID.csv | this is a mapping table that contains a unique identifier which links the customer’s demographic details |
| Transaction_ID.csv | this is a mapping table that contains transaction to customer mapping and payment mode |
| precipitation.csv | Rain depth (in) from 01/01/2016 to 12/31/2018 |

# Part 2: Code

## I: Import Data

In [1]:
import pandas as pd
import requests
import io
import numpy as np
from datetime import date, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Downloading the raw csv file from your GitHub account
cab_data = "https://raw.githubusercontent.com/DataGlacier/DataSets/main/Cab_Data.csv" 
city_data = "https://raw.githubusercontent.com/DataGlacier/DataSets/main/City.csv"
customer_data = "https://raw.githubusercontent.com/DataGlacier/DataSets/main/Customer_ID.csv"
transaction_data = "https://raw.githubusercontent.com/DataGlacier/DataSets/main/Transaction_ID.csv"

download_cab = requests.get(cab_data).content
download_city = requests.get(city_data).content
download_customer = requests.get(customer_data).content
download_transaction = requests.get(transaction_data).content

cab = pd.read_csv(io.StringIO(download_cab.decode('utf-8'))) # Reading the downloaded content and turning it into a pandas dataframe
city = pd.read_csv(io.StringIO(download_city.decode('utf-8')))
customer = pd.read_csv(io.StringIO(download_customer.decode('utf-8')))
transaction = pd.read_csv(io.StringIO(download_transaction.decode('utf-8')))

print("size of cab data is:", cab.shape) #there were 359392 total transactions
print("size of city data is:", city.shape) #there are 20 cities
print("size of customer data is:", customer.shape) #there are 49171 customers in the database
print("size of transaction data is:", transaction.shape) #there were 440098 transactions

size of cab data is: (359392, 7)
size of city data is: (20, 3)
size of customer data is: (49171, 4)
size of transaction data is: (440098, 3)


In [3]:
#EXTRA DATASET: precipitation data
precipitation = pd.read_csv("precipitation.csv")
precipitation.head(2)

,City,Date of Travel,Precipitation
0,ATLANTA GA,2016-01-09,NaN
1,ATLANTA GA,2016-01-10,0.35


In [4]:
cab.head(2)

,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip
0,10000011,42377,Pink Cab,ATLANTA GA,30.45,370.95,313.635
1,10000012,42375,Pink Cab,ATLANTA GA,28.62,358.52,334.854


In [5]:
city.head(2)

,City,Population,Users
0,NEW YORK NY,"8,405,837","302,149"
1,CHICAGO IL,"1,955,130","164,468"


In [6]:
customer.head(2)

,Customer ID,Gender,Age,Income (USD/Month)
0,29290,Male,28,10813
1,27703,Male,27,9237


In [7]:
transaction.head(2)

,Transaction ID,Customer ID,Payment_Mode
0,10000011,29290,Card
1,10000012,27703,Card


# II: Data Cleaning

In [8]:
#convert days since 01/01/1990 to %yyyy%mm%dd
days_ = []
start = date(1900,1,1)      # This is the "days since" part

for i in range(len(cab)):
    days = cab["Date of Travel"][i].tolist()               # This may work for floats in general, but using integers
    delta = timedelta(days)     # Create a time delta object from the number of days
    days_.append(start + delta)     # Add the specified number of days to 1900
    
cab["Date of Travel"] = days_
cab.head(2)

,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip
0,10000011,2016-01-10,Pink Cab,ATLANTA GA,30.45,370.95,313.635
1,10000012,2016-01-08,Pink Cab,ATLANTA GA,28.62,358.52,334.854


In [9]:
#change to datetime 
cab['Date of Travel'] = pd.to_datetime(cab['Date of Travel'])
precipitation['Date of Travel'] = pd.to_datetime(precipitation['Date of Travel'])

### Merge Datasets

In [10]:
#merges
merge1 = transaction.merge(cab, how = "left", on = "Transaction ID")
merge2 = merge1.merge(customer, how = "left", on = "Customer ID")
merge3 = merge2.merge(city, how = "left", on = "City")

In [11]:
#FINAL MERGE
final_df = merge3.merge(precipitation, left_on=['City','Date of Travel'],
           right_on=['City','Date of Travel'], 
           how='left')

#### Arrange columns

In [ ]:
#add columns

#profit column
final_df["Profit"] = final_df["Price Charged"] - final_df["Cost of Trip"]
#year column
final_df['Date of Travel'] = pd.to_datetime(final_df['Date of Travel'])
final_df['Year'] = final_df['Date of Travel'].dt.year

In [ ]:
#delete columns
delcol = ["Payment_Mode", "Users", "Population", "Price Charged", "Cost of Trip"]
final_df = final_df.drop(delcol, axis=1)

In [ ]:
neworder = ['Transaction ID', 'City', "Year", 'Date of Travel', "Precipitation", 'Company', 'Customer ID', "Gender", "Age", "Income (USD/Month)", "KM Travelled", "Profit"]
final_df = final_df.reindex(columns=neworder)

In [ ]:
#final_df['Year'] = final_df['Date of Travel'].dt.year

In [ ]:
final_df['Year'].unique()

#### Data Cleaning

In [ ]:
#drop duplicates
final_df.drop_duplicates()
print("Done")

In [ ]:
#delete rows with no cab data
options = ["Pink Cab", "Yellow Cab"]
final_df = final_df[final_df['Company'].isin(options)]

#delete rows with non-year data
options2 = [2016, 2017, 2018]
cab_data = final_df[final_df["Year"].isin(options2)]

In [ ]:
#final dataset
final_df.head(5)

# III: Data Analysis

**A1. What are the different cities?**

In [ ]:
print("The different cities are: ")
print(cab_data.City.unique())

**A2: How many trips per cab company?**

In [ ]:
cab_data.groupby("Company").count()["Transaction ID"]

Pink Cab company completed 930,349 trips and the Yellow Cab company completed 3,056,926 trips from 2016 to 2018.

**A3: What is the income of the customers?**

In [ ]:
sns.boxplot(x='Company', y='Income (USD/Month)', data=cab_data)

In [ ]:
print("Company: [Pink Cab | Yellow Cab]")
print("Minimum monthly income by:", cab_data.groupby("Company")["Income (USD/Month)"].min().values)
print("Average monthly income by:", cab_data.groupby("Company")["Income (USD/Month)"].mean().values)
print("Maximum monthly income by:", cab_data.groupby("Company")["Income (USD/Month)"].max().values)
print("Standard deviation of monthly income:", cab_data.groupby("Company")["Income (USD/Month)"].std().values)

Pink Cab customers have a higher mininum, average, and maximum income than the yellow cab customers.

**A4: Gender composition by cab company.**

In [ ]:
cab_data.groupby(["Company", "Gender"])["Transaction ID"].count()

Each company has more male customers than female customers.

**A5: City Composition**

In [ ]:
pink_1 = cab_data.groupby(["Company", "City"])["Transaction ID"].count()[0:19]
yellow_1 = cab_data.groupby(["Company", "City"])["Transaction ID"].count()[19:40]

In [ ]:
plt.scatter(cab_data.City.unique(), pink_1.values, label = "Pink", color = 'pink')
plt.scatter(cab_data.City.unique(), yellow_1.values, label = "Yellow", color = "gold")
plt.xticks(rotation = 75)
plt.legend()
plt.title("Number of Rides by City")
plt.show()

In [ ]:
#without atlanta
plt.scatter(cab_data.City.unique()[1:19], pink_1.values[1:19], label = "Pink", color = 'pink')
plt.scatter(cab_data.City.unique()[1:19], yellow_1.values[1:19], label = "Yellow", color = "gold")
plt.xticks(rotation = 75)
plt.legend()
plt.title("Number of Rides by City (Without Atlanta)")
plt.show()

**A6. Unique customers by City**

In [ ]:
pink_customers = cab_data[cab_data["Company"] == "Pink Cab"]
yellow_customers = cab_data[cab_data["Company"] == "Yellow Cab"]

In [ ]:
p = pink_customers.drop_duplicates().groupby("City", as_index=False)["Customer ID"].agg(["count"])
y = yellow_customers.drop_duplicates().groupby("City", as_index=False)["Customer ID"].agg(["count"])

In [ ]:
plt.scatter(p.index, p.values, label = "Pink Cab", color = "red")
plt.scatter(y.index, y.values, label = "Yellow Cab", color = "green")
plt.xticks(rotation = 75)
plt.legend()
plt.title("Number of Unique Customers by City")
plt.ylabel("Unique Users")
plt.show()

### **B. Profit Analysis**

**B1: Profit Table**

In [ ]:
#table of total profit, number of rides, and average profit per ride by company
profit = cab_data.groupby("Company")["Profit"].sum()
num_of_rides = cab_data.groupby("Company")["Transaction ID"].count()
av_ppr = cab_data.groupby("Company")["Profit"].mean()

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)
df = pd.DataFrame({"Company":["Pink Cab", "Yellow Cab"],
                   "Profit":[profit[0],profit[1]],
                   "Number of Rides": [num_of_rides[0], num_of_rides[1]],
                   "Average Profit Per Ride": [av_ppr[0], av_ppr[1]]
                  })
print(df)

**B2: Average Profit per km**

In [ ]:
#make a new column called "Profit/km"
pd.options.mode.chained_assignment = None  # default='warn'
cab_data["Profit/KM"] = cab_data.Profit/cab_data["KM Travelled"]

In [ ]:
p_km = cab_data.groupby(["Company", "Year"])["Profit/KM"].mean()
p_km

In [ ]:
import plotly.graph_objects as go
x = cab_data.Year.unique()
plot = go.Figure(
    data=[go.Bar(
        name = 'Pink Cab',
        x = x,
        y = [p_km[0], p_km[1], p_km[2]]),
          go.Bar(
              name = 'Yellow Cab', 
              x = x,
              y = [p_km[3], p_km[4], p_km[5]])
         ]
)

plot.update_xaxes(title_text="Year")
plot.update_yaxes(title_text="Average Profit/KM")
plot.show()

**B3. Profit by Year**

In [ ]:
#what is the cab with the most profit 
pink_2 = cab_data.groupby(["Company", "Year"])["Profit"].sum()[0:3]
yellow_2 = cab_data.groupby(["Company", "Year"])["Profit"].sum()[3:6]

In [ ]:
years = ["2016", "2017", "2018"]
plt.plot(years, pink_2.values, ls = '-', label = "Pink", color = 'pink')
plt.plot(years, yellow_2.values, ls = '-', label = "Yellow", color = "gold")
plt.legend()
plt.title("Total Profit by Year")
plt.ylabel("Total Profit (in millions)")
plt.show()

**B4: Profit Contribution by Class**

* Class 1: >= 15000 USD per Month 
* Class 2: 2500 USD < x < 15000 USD per Month
* Class 3: <= 2500 USH per Month

In [ ]:
def salary_stats(value):
    if value > 15000:
        return "Class 1"
    if 2500 <= value < 15000:
        return "Class 2"
    elif value < 2500:
        return "Class 3"
 
cab_data['Class'] = cab_data['Income (USD/Month)'].map(salary_stats)
display(cab_data.head(2))

In [ ]:
pbc = cab_data.groupby(["Company", "Class"])["Profit"].sum()
pbc

In [ ]:
x = ["Pink Cab", "Yellow Cab"]
plot = go.Figure(
    data=[go.Bar(
        name = 'Class 1',
        x = x,
        y = [pbc[0], pbc[3]]),
          go.Bar(
              name = 'Class 2', 
              x = x,
              y = [pbc[1], pbc[4]]),
          go.Bar(
              name = 'Class 3', 
              x = x,
              y = [pbc[2], pbc[5]])
         ]
)

plot.update_xaxes(title_text="Company")
plot.update_yaxes(title_text="Profit Contribution")
plot.show()

**B5. Profit Contribution by Gender**

In [ ]:
pbg = cab_data.groupby(["Company", "Gender"])["Profit"].sum()
pbg

In [ ]:
x = ["Pink Cab", "Yellow Cab"]
plot = go.Figure(
    data=[go.Bar(
        name = 'Female',
        x = x,
        y = [pbg[0], pbg[2]]),
          go.Bar(
              name = 'Male', 
              x = x,
              y = [pbg[1], pbg[3]])
         ]
)

plot.update_xaxes(title_text="Gender")
plot.update_yaxes(title_text="Profit Contribution")
plot.show()

### **C. Precipitation Insights**

In [ ]:
print("About", cab_data['Precipitation'].isnull().sum()/cab_data.size *100, "% of the precipitation data is missing. This is a good stat!")


In [ ]:
pink = cab_data[cab_data["Company"] == "Pink Cab"]
yellow = cab_data[cab_data["Company"] == "Yellow Cab"]

In [ ]:
#as precipitation increases, how many trips are taken
#histogram
plt.hist(yellow["Precipitation"], bins = 30, color = "green", label = "Yellow Cab")
plt.hist(pink["Precipitation"], bins = 30, color = "red", label = "Pink Cab")
plt.title("Precipitation Frequency")
plt.ylabel("Frequency (in millions)")
plt.xlabel("Precipitation Depth (in)")
plt.legend()
plt.show()

# Part 3. Recommendation

* **Customer Income:** Both companies similar maximum, minimum, and average customer income.

* **Trip Quantities:** Yellow cab completed more trips and has more distinct customers than the Pink cab. There is a greater difference in cities like New York City, Atlanta, Washington DC, and Chicago.

* **Average Profit per KM:** Yellow cab’s average profit per KM is almost three times the average profit per KM of the Pink cab.

* **Income wise Reach:** Both the cabs are very popular in high and medium income class but here also Yellow cab is performing better than Pink cab in offering their services to all the three income class group (Class 1, Class 2 and Class 3)

* **Precipitation:** Customers are most likely going to use Yellow cab services than Pink cab on a rainy day. Even so, they choose Yellow cab on non-rainy days.

**Invest in Yellow Cab due to its high customer base, profit earnings, and reliability.**